In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
#using model.save_weights() to save the weights of the model in HDF5 format
# model.save_weights("/content/gdrive/My Drive/weights.h5")

Mounted at /content/gdrive


In [2]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops

from datasets import load_dataset
from random import randrange

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM

from trl import SFTTrainer

from huggingface_hub import login

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.8/100.8 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.1 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [4]:
model_name = "Salesforce/codegen-350M-mono"
dataset_name = "iamtarun/python_code_instructions_18k_alpaca"
device_map = {"": 0}

In [5]:
peft_config = LoraConfig(
      lora_alpha=16,
      lora_dropout=0.1,
      r=64,
      bias="none",
      task_type="CAUSAL_LM",
)

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

In [7]:
from huggingface_hub import notebook_login
# Log in to HF Hub
notebook_login()

In [8]:
def prompt_instruction_format(sample):
  return f"""### Instruction:
    Use the Task below and the Input given to write the Response, which is a programming code that can solve the following Task:

    ### Task:
    {sample['instruction']}

    ### Input:
    {sample['input']}

    ### Response:
    {sample['output']}
    """

In [10]:
code_dataset = load_dataset(dataset_name, split='train[:5000]')

# code_dataset = load_dataset(dataset_name, split=split)


model = AutoModelForCausalLM.from_pretrained(model_name,
          quantization_config=bnb_config,
          use_cache = False,
          device_map=device_map)
model.config.pretraining_tp = 1

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

In [13]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
trainingArgs = TrainingArguments(
    output_dir="./colab_code_generator_FT_code_gen_UT",
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=5,
    save_strategy="epoch",
    learning_rate=2e-4,
    weight_decay=0.001,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=False,
    lr_scheduler_type="cosine",
    disable_tqdm=True,
    seed=42
)



trainer = SFTTrainer(
    model=model,
    train_dataset=code_dataset,
    peft_config=peft_config,
    max_seq_length=2048,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=prompt_instruction_format,
    args=trainingArgs,
)

In [15]:
trainer.train()

Token indices sequence length is longer than the specified maximum sequence length for this model (2905 > 2048). Running this sequence through the model will result in indexing errors
You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.1666, 'learning_rate': 2.6315789473684212e-05, 'epoch': 0.01}
{'loss': 1.174, 'learning_rate': 5.2631578947368424e-05, 'epoch': 0.02}
{'loss': 1.1239, 'learning_rate': 7.894736842105263e-05, 'epoch': 0.02}
{'loss': 1.0462, 'learning_rate': 0.00010526315789473685, 'epoch': 0.03}
{'loss': 0.9808, 'learning_rate': 0.00013157894736842108, 'epoch': 0.04}
{'loss': 0.9396, 'learning_rate': 0.00015789473684210527, 'epoch': 0.05}
{'loss': 0.9461, 'learning_rate': 0.00018421052631578948, 'epoch': 0.06}
{'loss': 0.8908, 'learning_rate': 0.00019999865623437013, 'epoch': 0.06}
{'loss': 0.816, 'learning_rate': 0.00019998353928577919, 'epoch': 0.07}
{'loss': 0.7835, 'learning_rate': 0.00019995162822919883, 'epoch': 0.08}
{'loss': 0.7965, 'learning_rate': 0.00019990292842468868, 'epoch': 0.09}
{'loss': 0.7807, 'learning_rate': 0.00019983744805229296, 'epoch': 0.1}
{'loss': 0.8367, 'learning_rate': 0.00019975519811066663, 'epoch': 0.1}
{'loss': 0.7388, 'learning_rate': 0.0001996561924152278,

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.786, 'learning_rate': 0.0001995404475958373, 'epoch': 1.0}
{'loss': 0.7248, 'learning_rate': 0.00019940798309400526, 'epoch': 1.01}
{'loss': 0.659, 'learning_rate': 0.00019925882115962568, 'epoch': 1.02}
{'loss': 0.7834, 'learning_rate': 0.00019909298684723904, 'epoch': 1.03}
{'loss': 0.7091, 'learning_rate': 0.000198910508011824, 'epoch': 1.03}
{'loss': 0.6875, 'learning_rate': 0.00019871141530411853, 'epoch': 1.04}
{'loss': 0.7292, 'learning_rate': 0.00019849574216547171, 'epoch': 1.05}
{'loss': 0.6792, 'learning_rate': 0.00019826352482222638, 'epoch': 1.06}
{'loss': 0.7476, 'learning_rate': 0.0001980148022796345, 'epoch': 1.07}
{'loss': 0.6446, 'learning_rate': 0.00019774961631530545, 'epoch': 1.07}
{'loss': 0.6659, 'learning_rate': 0.00019746801147218842, 'epoch': 1.08}
{'loss': 0.7153, 'learning_rate': 0.00019717003505109095, 'epoch': 1.09}
{'loss': 0.7151, 'learning_rate': 0.00019685573710273376, 'epoch': 1.1}
{'loss': 0.782, 'learning_rate': 0.00019652517041934356, 'e

TrainOutput(global_step=148, training_loss=0.8149129047587111, metrics={'train_runtime': 2697.457, 'train_samples_per_second': 3.707, 'train_steps_per_second': 0.463, 'train_loss': 0.8149129047587111, 'epoch': 1.12})

In [16]:
trainer.save_model('./saved')

In [17]:
trained_model = AutoModelForCausalLM.from_pretrained('./saved')

In [18]:
trained_tokenizer = AutoTokenizer.from_pretrained('./saved')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
# merged = trained_model.merge_and_unload()
# merged.save_pretrained("merged",safe_serialization=True)
# tokenizer.save_pretrained("merged")

#push merged model to the hub
trained_model.push_to_hub("01GangaPutraBheeshma/colab_code_generator_FT_code_gen_UT")
trained_tokenizer.push_to_hub("01GangaPutraBheeshma/colab_code_generator_FT_code_gen_UT")

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:389: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/01GangaPutraBheeshma/colab_code_generator_FT_code_gen_UT/commit/9a228231bbce763a14da7e007f6bddb24f402561', commit_message='Upload tokenizer', commit_description='', oid='9a228231bbce763a14da7e007f6bddb24f402561', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# import math

# eval_results = trainer.evaluate()
# print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [20]:
instruction="Write a Python program to generate a binary search  given an array input and element to be searched."
input="[2,3,4,5], [5]"

prompt = f"""### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
{instruction}

### Input:
{input}

### Response:
"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids
# with torch.inference_mode():

print(f"-------------------------\n\n")
print(f"Prompt:\n{prompt}\n")
print(f"-------------------------\n\n")

print(f"Before Training Response :")
output_before = model.generate(input_ids=input_ids, max_new_tokens=200, do_sample=True, top_p=0.9,temperature=0.5)
print(f"Generated instruction:\n{tokenizer.batch_decode(output_before.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n\n")

print(f"After Training Response :")
outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=200, do_sample=True, top_p=0.9,temperature=0.5)

print(f"-------------------------\n\n")
print(f"Generated instruction:\n{trained_tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n\n")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------------------------


Prompt:
### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
Write a Python program to generate a binary search  given an array input and element to be searched.

### Input:
[2,3,4,5], [5]

### Response:


-------------------------


Before Training Response :


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1591: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkp

Generated instruction:
# Generate a binary search function
def binary_search(array, target):
    # Initialize the middle point
    mid = len(array) // 2

    # Check if the target is present at the middle point
    if array[mid] == target:
        return mid
    elif array[mid] > target:
        return binary_search(array[:mid], target)
    else:
        return binary_search(array[mid:], target)

# Generate an array of elements
array = [2, 3, 4, 5]

# Search for the target element
target = 5

# Find the middle point
mid = len(array) // 2

# Find the element at the middle point
result = binary_search(array, target)

# Print the result
print(result)
    
-------------------------


After Training Response :
-------------------------


Generated instruction:
def binary_search(array, element):
    start = 0
    end = len(array) - 1
    while start <= end:
        mid = (start + end) // 2
        if array[mid] == element:
            return mid
        elif array[mid] > element:
           

In [23]:
instruction="Write a Python program to reverse a binary tree."
input=""

prompt = f"""### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
{instruction}

### Input:
{input}

### Response:
"""

test_model_UT = AutoModelForCausalLM.from_pretrained("01GangaPutraBheeshma/colab_code_generator_FT_code_gen_UT")
test_tokenizer_UT = AutoTokenizer.from_pretrained("01GangaPutraBheeshma/colab_code_generator_FT_code_gen_UT")


input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids

trained_input_ids = test_tokenizer_UT(prompt, return_tensors="pt", truncation=True).input_ids

# with torch.inference_mode():

print(f"-------------------------\n\n")
print(f"Prompt:\n{prompt}\n")
print(f"-------------------------\n\n")

print(f"Before Training Response :")
output_before = model.generate(input_ids=input_ids, max_new_tokens=200, do_sample=True, top_p=0.9,temperature=0.5)
print(f"Generated instruction:\n{tokenizer.batch_decode(output_before.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n\n")

print(f"After Training Response :")
outputs = test_model_UT.generate(input_ids=trained_input_ids, max_new_tokens=500, do_sample=True, top_p=0.9,temperature=0.5)

print(f"-------------------------\n\n")
print(f"Generated instruction:\n{test_tokenizer_UT.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n\n")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------------------------


Prompt:
### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
Write a Python program to reverse a binary tree.

### Input:


### Response:


-------------------------


Before Training Response :


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated instruction:

# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def invertTree(self, root: TreeNode) -> TreeNode:
        if not root:
            return None
        
        root.left, root.right = self.invertTree(root.right), self.invertTree(root.left)
        
        return root
    
-------------------------


After Training Response :
-------------------------


Generated instruction:
class Node:
    def __init__(self, data):
        self.data = data
        self.left = None
        self.right = None

def reverseBinaryTree(root):
    if root is None:
        return None
    
    left = reverseBinaryTree(root.left)
    right = reverseBinaryTree(root.right)

    root.left = right
    root.right = left

    return root

root = Node(1)
root.left = Node(2)
root.right = Node(3)

print(reverseBinaryTree(root))
    
-

In [25]:
instruction="Write a Python Script to search a filename within a list of filenames and provide me the time complexity of the algorithm"
input=""

prompt = f"""### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
{instruction}

### Input:
{input}

### Response:
"""

test_model_UT = AutoModelForCausalLM.from_pretrained("01GangaPutraBheeshma/colab_code_generator_FT_code_gen_UT")
test_tokenizer_UT = AutoTokenizer.from_pretrained("01GangaPutraBheeshma/colab_code_generator_FT_code_gen_UT")


input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids

trained_input_ids = test_tokenizer_UT(prompt, return_tensors="pt", truncation=True).input_ids

# with torch.inference_mode():

print(f"-------------------------\n\n")
print(f"Prompt:\n{prompt}\n")
print(f"-------------------------\n\n")

print(f"Before Training Response :")
output_before = model.generate(input_ids=input_ids, max_new_tokens=200, do_sample=True, top_p=0.9,temperature=0.5)
print(f"Generated instruction:\n{tokenizer.batch_decode(output_before.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n\n")

print(f"After Training Response :")
outputs = test_model_UT.generate(input_ids=trained_input_ids, max_new_tokens=500, do_sample=True, top_p=0.9,temperature=0.5)

print(f"-------------------------\n\n")
print(f"Generated instruction:\n{test_tokenizer_UT.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}")
print(f"-------------------------\n\n")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-------------------------


Prompt:
### Instruction:
Use the Task below and the Input given to write the Response, which is a programming code that can solve the Task.

### Task:
Write a Python Script to search a filename within a list of filenames and provide me the time complexity of the algorithm

### Input:


### Response:


-------------------------


Before Training Response :


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated instruction:

def search_time(filenames):
    filenames = [f for f in filenames if f.endswith(".txt")]
    return len(filenames)

filenames = ["sample.txt", "sample2.txt", "sample3.txt"]

print(search_time(filenames))
    
-------------------------


After Training Response :
-------------------------


Generated instruction:

import os
import time

# Get the list of filenames
filenames = os.listdir('.')

# Loop through the filenames
for filename in filenames:
    # Check if the filename contains the word 'time'
    if 'time' in filename:
        # Get the time of the file
        time_of_file = os.path.getctime(filename)
        # Print the time of the file
        print(time_of_file)
    
-------------------------


